## Importing important libraries

In [27]:
# importing all the libraries needed 
import numpy as np
from sklearn.externals import joblib
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
import matplotlib.dates as mdates
%matplotlib inline
import datetime
import statsmodels.tsa.api as smt
sns.set_style("whitegrid")
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error, mean_absolute_error

#For date time functions
from datetime import datetime
from datetime import timedelta
import math
import datetime
# Importing the most popular regression libraries.
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression, ridge_regression, Lasso, SGDRegressor, Ridge
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor,StackingRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

## Function -1
#### For generating prediction value on raw data

In [34]:
def generating_prediction(final_data):    
    final_data.loc[final_data.MarkDown1.isnull() ,'MarkDown1']= 0
    final_data.loc[final_data.MarkDown2.isnull() ,'MarkDown2']= 0
    final_data.loc[final_data.MarkDown3.isnull() ,'MarkDown3']= 0
    final_data.loc[final_data.MarkDown4.isnull() ,'MarkDown4']= 0
    final_data.loc[final_data.MarkDown5.isnull() ,'MarkDown5']= 0
      
    final_data['Date'] = pd.to_datetime(final_data['Date'])
    final_data['Year'] = final_data['Date'].dt.year
    final_data['Month']= final_data['Date'].dt.month
    final_data['Week'] = final_data['Date'].dt.week
    final_data['Day']  = final_data['Date'].dt.day

    dates =[]
    for ptr in holidays.US(years = 2010).items():
        dates.append(ptr[0])
    for ptr in holidays.US(years = 2011).items():
        dates.append(ptr[0])
    for ptr in holidays.US(years = 2012).items():
        dates.append(ptr[0])
    for ptr in holidays.US(years = 2013).items():
        dates.append(ptr[0])
        
    holiday_count=[] 
    for index, row in final_data.iterrows():
        dat = final_data['Date'][index]
        dt=[]
        for i in range(0,5):
            dt.append(dat - datetime.timedelta(days = i))
        for i in range(1,3):
            dt.append(dat + datetime.timedelta(days = i))
        count = 0
        for date in dates:
            if date in dt:
                count +=1
        holiday_count.append(count)
    final_data['Holidays'] = np.array(holiday_count)

    final_data.loc[(final_data.IsHoliday==True) ,'IsHoliday']= 1
    final_data.loc[(final_data.IsHoliday==False) ,'IsHoliday']= 0

    final_data.loc[(final_data.Type=='A') ,'Type']= 1
    final_data.loc[(final_data.Type=='B') ,'Type']= 2
    final_data.loc[(final_data.Type=='C') ,'Type']= 3
    final_data=final_data.reset_index(drop=True)
    final_data=final_data[['Store','Dept','IsHoliday','Size','Week','Type','Year','Holidays','Day']]
    
    Regressor = joblib.load('model.pkl')
    pred = Regressor.predict(final_data)
    
    return pred

In [37]:
train_data=pd.read_csv('train.csv',parse_dates=True)
features_data=pd.read_csv('features.csv',parse_dates=True)
stores_data=pd.read_csv('stores.csv')
stores = stores_data.merge(features_data, on ='Store' , how = 'left')
final_data_train = train_data.merge(stores, on = ['Store', 'Date', 'IsHoliday'], how = 'left')
y = final_data_train['Weekly_Sales']
X = final_data_train.drop(['Weekly_Sales'], axis=1) 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

import random
data_for_prediction = X_test.sample()
print('Prediction of eekly sales for given raw data is : ',generating_prediction(data_for_prediction))


Prediction of eekly sales for given raw data is :  [816.39535]


## Function 2
#### For getting WMAE value

In [38]:
def generating_Wmae_score(final_data):    
    final_data.loc[final_data.MarkDown1.isnull() ,'MarkDown1']= 0
    final_data.loc[final_data.MarkDown2.isnull() ,'MarkDown2']= 0
    final_data.loc[final_data.MarkDown3.isnull() ,'MarkDown3']= 0
    final_data.loc[final_data.MarkDown4.isnull() ,'MarkDown4']= 0
    final_data.loc[final_data.MarkDown5.isnull() ,'MarkDown5']= 0
      
    final_data['Date'] = pd.to_datetime(final_data['Date'])
    final_data['Year'] = final_data['Date'].dt.year
    final_data['Month']= final_data['Date'].dt.month
    final_data['Week'] = final_data['Date'].dt.week
    final_data['Day']  = final_data['Date'].dt.day

    dates =[]
    for ptr in holidays.US(years = 2010).items():
        dates.append(ptr[0])
    for ptr in holidays.US(years = 2011).items():
        dates.append(ptr[0])
    for ptr in holidays.US(years = 2012).items():
        dates.append(ptr[0])
    for ptr in holidays.US(years = 2013).items():
        dates.append(ptr[0])
    holiday_count=[] 
    for index, row in final_data.iterrows():
        dat = final_data['Date'][index]
        dt=[]
        for i in range(0,5):
            dt.append(dat - datetime.timedelta(days = i))
        for i in range(1,3):
            dt.append(dat + datetime.timedelta(days = i))
        count = 0
        for date in dates:
            if date in dt:
                count +=1
        holiday_count.append(count)
    final_data['Holidays'] = np.array(holiday_count)

    final_data.loc[(final_data.IsHoliday==True) ,'IsHoliday']= 1
    final_data.loc[(final_data.IsHoliday==False) ,'IsHoliday']= 0

    final_data.loc[(final_data.Type=='A') ,'Type']= 1
    final_data.loc[(final_data.Type=='B') ,'Type']= 2
    final_data.loc[(final_data.Type=='C') ,'Type']= 3
    final_data=final_data.reset_index(drop=True)
    y_true=final_data['Weekly_Sales']
    final_data=final_data[['Store','Dept','IsHoliday','Size','Week','Type','Year','Holidays','Day']]
    
    Regressor = joblib.load('model.pkl')
    y_pred = Regressor.predict(final_data)
  
    weights = final_data.IsHoliday.apply(lambda x: 5 if x else 1)
    return np.round(np.sum(weights*abs(y_true-y_pred))/(np.sum(weights)), 2)

In [39]:
train_data=pd.read_csv('train.csv',parse_dates=True)
features_data=pd.read_csv('features.csv',parse_dates=True)
stores_data=pd.read_csv('stores.csv')
stores = stores_data.merge(features_data, on ='Store' , how = 'left')
final_data_train = train_data.merge(stores, on = ['Store', 'Date', 'IsHoliday'], how = 'left') 
Train,Test = train_test_split(final_data_train,test_size=0.3)

import random
data_for_prediction = Test.sample()
print('Wmae score for the given raw data is :',generating_Wmae_score(data_for_prediction))

Wmae score for the given raw data is : 5630.31
